<span STYLE="font-size:150%"> 
    Compute segment volumes from microCT scans
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 15 November 2024

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume/microCT_volume.nrrd'
output_dir_path = 'segmented_volumes'
file_bone_segment = 'segmented_volumes/Bone.seg.nrrd'
file_roi_segment = 'segmented_volumes/ROI.seg.nrrd'

In [ ]:
path = Path(volume_file)

# Remove image numbering _0000, _0001 ...
filename_output = path.stem[:-4]

In [ ]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Make ```segmented_volumes``` folder

In [ ]:
output_path = Path(output_dir_path)

output_path.mkdir(parents=True, exist_ok=True)

Get norebook directory name

In [ ]:
sample_name = sample_dir.rstrip('/').split('/')[-1]

# Create segmentationNode for `Bone`

## Create segmentation-related nodes

Create segmentation node

In [ ]:
segmentationNode = ps.segmentation.segmentationNode(name='Segmentation')

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Load segmentation to segmentation node

Load segmentation from .nrrd file as labelmap node slicer util module to load from File [github](https://github.com/Slicer/Slicer/blob/master/Base/Python/slicer/util.py#L341-L344)

In [ ]:
# Load segmentation from .seg.nrrd file (includes segment names and colors)

bone_labelmap = slicer.util.loadNodeFromFile(file_bone_segment, 'VolumeFile', properties={'labelmap':True})

Import bone labelmapNode to segmentationNode

In [ ]:
slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(bone_labelmap, segmentationNode)

Rename segment of the segmentation node 

In [ ]:
segmentation = segmentationNode.GetSegmentation()
segment = segmentation.GetNthSegment(0)
segment.SetName("Bone")

Delete bone labelmapNode

In [ ]:
slicer.mrmlScene.RemoveNode(bone_labelmap)
# masterVolumeNode.SetDisplayVisibility(1)

## Load ROI segmentation to segmentation node

Load segmentation from .nrrd file as labelmap node slicer util module to load from File [github](https://github.com/Slicer/Slicer/blob/master/Base/Python/slicer/util.py#L341-L344)

In [ ]:
# Load segmentation from .seg.nrrd file (includes segment names and colors)

roi_labelmap = slicer.util.loadNodeFromFile(file_roi_segment, 'VolumeFile', properties={'labelmap':True})
# segmentationNode = slicer.util.loadSegmentation(file_nuclei_segmentation)

Import bone labelmapNode to segmentationNode

In [ ]:
slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(roi_labelmap, segmentationNode)

Rename segment of the segmentation node 

In [ ]:
segmentation = segmentationNode.GetSegmentation()
segment = segmentation.GetNthSegment(1)
segment.SetName("ROI")

Delete labelmapNode

In [ ]:
slicer.mrmlScene.RemoveNode(roi_labelmap)
# masterVolumeNode.SetDisplayVisibility(1)

## Create new segment copy

In [ ]:
copy_segmentName = 'Bone'

segmentation = segmentationNode.GetSegmentation()
copy_segmentID = segmentation.GetSegmentIdBySegmentName(copy_segmentName)

In [ ]:
new_segmentName = 'Bone_ROI'

In [ ]:
# '''
# COPY operation from the [SegmentEditorLogicalEffect](https://github.com/Slicer/Slicer/blob/4483cc0e6f288b0816b6329f1829d9ef8c5aa81a/Modules/Loadable/Segmentations/EditorEffects/Python/SegmentEditorLogicalEffect.py)
# '''
# 

# Create segment
addedSegmentID = segmentationNode.GetSegmentation().AddEmptySegment(new_segmentName)
segmentEditorNode.SetSelectedSegmentID(addedSegmentID)

# Fill by thresholding
segmentEditorWidget.setActiveEffectByName("Logical operators")
effect = segmentEditorWidget.activeEffect()
effect.setParameter('Operation','COPY')
effect.setParameter("ModifierSegmentID",copy_segmentID)
effect.self().onApply()

Set segment colors

In [ ]:
segments_color = {'Bone':(0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
                  'ROI':(230/255, 75/255, 53/255),
                  'Bone_ROI':(77/255, 187/255, 213/255)
                  }

ps.segmentation.set_segments_color(segments_color, segmentationNode)

# Operation on segments

## Manual fix of the segmentation

Sometimes it might be necessary to remove speckles at the image boundaries. If so, use the `scissor` tool in the `Segment Editor` before proceeding with the rest of the script. 

## Intersect cylinder segments with bone segment

In [ ]:
# ps.segmentation.logical_intersect('1_1', '1', segmentationNode, segmentEditorNode, segmentEditorWidget)

In [ ]:
# Function to get the segment ID by name
def getSegmentIDByName(segmentationNode, segmentName):
    segmentation = segmentationNode.GetSegmentation()
    for i in range(segmentation.GetNumberOfSegments()):
        segmentID = segmentation.GetNthSegmentID(i)
        segment = segmentation.GetSegment(segmentID)
        if segment.GetName() == segmentName:
            return segmentID
    return None  # Return None if no segment with the given name is found

# Define the names of the segments
masterSegmentName = "Bone_ROI"      # Replace with the actual name of the master segment
modifierSegmentName = "ROI"  # Replace with the actual name of the modifier segment

# Get the segment IDs by their names
masterSegmentID = getSegmentIDByName(segmentationNode, masterSegmentName)
modifierSegmentID = getSegmentIDByName(segmentationNode, modifierSegmentName)

if not masterSegmentID:
    raise RuntimeError(f"Master segment '{masterSegmentName}' not found.")

if not modifierSegmentID:
    raise RuntimeError(f"Modifier segment '{modifierSegmentName}' not found.")

# Set the master segment as the current segment
segmentEditorNode.SetSelectedSegmentID(masterSegmentID)

# Activate the Logical operators effect
segmentEditorWidget.setActiveEffectByName("Logical operators")
effect = segmentEditorWidget.activeEffect()

if not effect:
    raise RuntimeError("Failed to activate the 'Logical operators' effect.")

# Configure the intersection operation
effect.setParameter("Operation", "INTERSECT")
effect.setParameter("ModifierSegmentID", modifierSegmentID)

# Apply the operation
effect.self().onApply()

print("Logical intersection operation completed successfully.")

# Compute bone ingrowth volumes

Compute segment statistics

In [ ]:
stats = ps.segmentation.segment_statistics(segmentationNode)

Store volume of each segment

In [ ]:
segment_names = []
volumes_ingrowth_mm3 = []

# Display volume of each segment
for segmentId in stats["SegmentIDs"]:
    
    volume_mm3 = stats[segmentId,"LabelmapSegmentStatisticsPlugin.volume_mm3"]
    volumes_ingrowth_mm3.append(volume_mm3)
    
    segmentName = segmentationNode.GetSegmentation().GetSegment(segmentId).GetName()
    segment_names.append(segmentName)

data_dict = {'segmentName': segment_names, 'volume_mm3':volumes_ingrowth_mm3}
df = pd.DataFrame(data_dict)
df

Add sample name columns to DataFrame

In [ ]:
df['sample'] = [sample_name] * len(df. index)
df

In [ ]:
outputfile = output_path / (sample_name + '_volumes.csv')
df.to_csv(outputfile, index=False)